In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
train_dir = 'flower-data/train/'
valid_dir = 'flower-data/val/'
test_dir = 'flower-data/test/'

In [4]:
batch_size = 32
img_sz = 256
channels = 3

In [5]:
datagenerator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True,

)

train_gen = datagenerator.flow_from_directory(
    directory=train_dir,
    target_size=(img_sz,img_sz),
    batch_size=batch_size,
    class_mode='sparse',
)
valid_gen = datagenerator.flow_from_directory(
    directory=valid_dir,
    target_size=(img_sz,img_sz),
    batch_size=batch_size,
    class_mode='sparse',
)

Found 45492 images belonging to 104 classes.
Found 12157 images belonging to 104 classes.


In [6]:
train_gen.class_indices.keys()

dict_keys(['alpine sea holly', 'anthurium', 'artichoke', 'azalea', 'balloon flower', 'barberton daisy', 'bee balm', 'bird of paradise', 'bishop of llandaff', 'black-eyed susan', 'blackberry lily', 'blanket flower', 'bolero deep blue', 'bougainvillea', 'bromelia', 'buttercup', 'californian poppy', 'camellia', 'canna lily', 'canterbury bells', 'cape flower', 'carnation', 'cautleya spicata', 'clematis', 'colt foot', 'columbine', 'common dandelion', 'common tulip', 'corn poppy', 'cosmos', 'cyclamen', 'daffodil', 'daisy', 'desert-rose', 'fire lily', 'foxglove', 'frangipani', 'fritillary', 'garden phlox', 'gaura', 'gazania', 'geranium', 'giant white arum lily', 'globe thistle', 'globe-flower', 'grape hyacinth', 'great masterwort', 'hard-leaved pocket orchid', 'hibiscus', 'hippeastrum', 'iris', 'japanese anemone', 'king protea', 'lenten rose', 'lilac hibiscus', 'lotus', 'love in the mist', 'magnolia', 'mallow', 'marigold', 'mexican petunia', 'monkshood', 'moon orchid', 'morning glory', 'orang

In [8]:
num_of_classes = len(os.listdir(train_dir))
print(num_of_classes)
class_names = list(train_gen.class_indices.keys())
print(class_names)

104
['alpine sea holly', 'anthurium', 'artichoke', 'azalea', 'balloon flower', 'barberton daisy', 'bee balm', 'bird of paradise', 'bishop of llandaff', 'black-eyed susan', 'blackberry lily', 'blanket flower', 'bolero deep blue', 'bougainvillea', 'bromelia', 'buttercup', 'californian poppy', 'camellia', 'canna lily', 'canterbury bells', 'cape flower', 'carnation', 'cautleya spicata', 'clematis', 'colt foot', 'columbine', 'common dandelion', 'common tulip', 'corn poppy', 'cosmos', 'cyclamen', 'daffodil', 'daisy', 'desert-rose', 'fire lily', 'foxglove', 'frangipani', 'fritillary', 'garden phlox', 'gaura', 'gazania', 'geranium', 'giant white arum lily', 'globe thistle', 'globe-flower', 'grape hyacinth', 'great masterwort', 'hard-leaved pocket orchid', 'hibiscus', 'hippeastrum', 'iris', 'japanese anemone', 'king protea', 'lenten rose', 'lilac hibiscus', 'lotus', 'love in the mist', 'magnolia', 'mallow', 'marigold', 'mexican petunia', 'monkshood', 'moon orchid', 'morning glory', 'orange dahl

In [9]:
count=0
for image_batch, label_batch in train_gen:
    print(image_batch[0])
    break

[[[0.30967763 0.23780325 0.11494847]
  [0.3458953  0.26118293 0.13059147]
  [0.38183692 0.27415892 0.13083592]
  ...
  [0.34773028 0.25882354 0.11240938]
  [0.34901804 0.25882354 0.11176551]
  [0.35030577 0.25882354 0.11112165]]

 [[0.3019512  0.23136458 0.10979753]
  [0.34074435 0.25860748 0.12930374]
  [0.37539825 0.27222732 0.13147978]
  ...
  [0.3491525  0.25503483 0.10854103]
  [0.3472209  0.25310323 0.10789716]
  [0.3452893  0.25117165 0.10725329]]

 [[0.2942248  0.2249259  0.1046466 ]
  [0.33559343 0.256032   0.128016  ]
  [0.36895958 0.27029574 0.13212366]
  ...
  [0.3399668  0.24584916 0.10467267]
  [0.33932292 0.24520528 0.10402881]
  [0.33867908 0.24456142 0.10338494]]

 ...

 [[0.0926935  0.12798762 0.04563467]
  [0.09204963 0.12734376 0.04499081]
  [0.09140577 0.12669988 0.04434694]
  ...
  [0.42085713 0.36324328 0.20304401]
  [0.41987363 0.35041538 0.22140223]
  [0.43266192 0.35554665 0.25092682]]

 [[0.08745421 0.12686113 0.04039539]
  [0.08616648 0.127505   0.03910765]


In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
epochs = 10
checkpoint =tf.keras.callbacks.ModelCheckpoint(f'model{epochs}epochs.h5',
                                               monitor='val_loss',
                                               mode='min',
                                               save_best_only=True,
                                               verbose=1)

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             min_delta=0,
                                             patience=50,
                                             verbose=1,restore_best_weights=True)

callbacks = [checkpoint,earlystop]

In [17]:
model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_sz,img_sz, 3)),
  layers.MaxPooling2D((2,2)),

  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),

  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_of_classes, activation='softmax')
])

In [18]:
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 64)       0

In [22]:

history = model.fit(
    train_gen,
    validation_data = valid_gen,
    epochs=epochs,
    steps_per_epoch = train_gen.samples//batch_size,
    validation_steps = valid_gen.samples//batch_size,
    callbacks=callbacks
)

Epoch 1/10
1421/1421 [==============================] - ETA: 0s - loss: 2.9449 - accuracy: 0.2619
Epoch 1: val_loss improved from inf to 2.59954, saving model to model10epochs.h5
1421/1421 [==============================] - 1147s 801ms/step - loss: 2.9449 - accuracy: 0.2619 - val_loss: 2.5995 - val_accuracy: 0.3265
Epoch 2/10
1421/1421 [==============================] - ETA: 0s - loss: 1.8089 - accuracy: 0.5123
Epoch 2: val_loss improved from 2.59954 to 2.48207, saving model to model10epochs.h5
1421/1421 [==============================] - 1131s 796ms/step - loss: 1.8089 - accuracy: 0.5123 - val_loss: 2.4821 - val_accuracy: 0.3817
Epoch 3/10
1421/1421 [==============================] - ETA: 0s - loss: 1.1996 - accuracy: 0.6686
Epoch 3: val_loss did not improve from 2.48207
1421/1421 [==============================] - 1366s 961ms/step - loss: 1.1996 - accuracy: 0.6686 - val_loss: 2.6462 - val_accuracy: 0.4100
Epoch 4/10
1421/1421 [==============================] - ETA: 0s - loss: 0.8149 

In [24]:
model.evaluate(valid_gen)

380/380 [==============================] - 198s 521ms/step - loss: 4.8301 - accuracy: 0.3919


[4.830072402954102, 0.39187300205230713]